Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [6]:
import requests
import pandas as pd
import numpy as np

In [13]:
van_bike_data = pd.read_csv('../data/van_bike_data.csv', encoding='unicode_escape')

In [8]:
van_bar_data=pd.read_csv("van_bar_data.csv",encoding="unicode_escape")

In [9]:
bars_yelp=pd.read_csv('bars_yelp.csv',encoding="unicode_escape")

In [10]:
Bike_Foursquare = pd.merge(van_bike_data,van_bar_data, on=["ll", "ll"])

In [11]:
Bike_Foursquare_Yelp=pd.merge(Bike_Foursquare,bars_yelp, on=["ll", "ll"])
Bike_Foursquare_Yelp

,free_bikes,name,ll,no of bars,avg_rating,avg_review,avg_distance,no_of_bars_yelp
0,21,10th & Cambie,"49.262487,-123.114397",50,3.70,103.89,889.53,27
1,9,Yaletown-Roundhouse Station,"49.274566,-123.121817",50,3.59,105.70,538.35,50
2,13,Dunsmuir & Beatty,"49.279764,-123.110154",50,3.91,148.94,573.26,50
3,2,12th & Yukon (City Hall),"49.260599,-123.113504",50,3.76,84.45,901.41,29
4,9,8th & Ash,"49.264215,-123.117772",50,3.45,138.09,640.18,11
...,...,...,...,...,...,...,...,...
236,11,Heather & 29th,"49.245535,-123.120496",4,4.00,1.00,942.16,1
237,16,Cardero & Robson,"49.289255,-123.132677",50,3.62,143.18,748.56,50
238,0,Commercial & 20th,"49.252656,-123.067965",11,0.00,0.00,0.00,0
239,2,Hornby & Drake,"49.277527,-123.129464",50,3.55,114.24,534.90,50


In [14]:
Bike_Foursquare_Yelp.to_csv('../data/Bike_Foursquare_Yelp.csv',index=False)

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [2]:
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [3]:
connection = create_connection("../data/sm_app.sqlite")
connection

Connection to SQLite DB successful


Look at the data before and after the join to validate your data.